In [1]:
import torch
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from PIL import Image
import h5py,io
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler  
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import io
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score
import gc
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score, precision_recall_curve
import gc
import xgboost as xgb
from xgboost.callback import TrainingCallback  # Import the base class
from sklearn.model_selection import GridSearchCV
from tqdm.notebook import tqdm
import torch.profiler as profiler
import os
from sklearn.preprocessing import StandardScaler

from joblib import Parallel, delayed
import torchvision
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import OrdinalEncoder
import sklearn
from sklearn.impute import KNNImputer
import pickle
from torch.nn.parallel import DataParallel
import os
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-tokenizer')  # Adjust the path if needed

In [4]:
batch_size =100

In [5]:
root = Path('/kaggle/input/isic-2024-challenge')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
sampling_ratio = 0.01
seed = 42

num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                  # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',            # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',         # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',      # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',# tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
    'contrast_shape',                #tbp_lv_deltaLBnorm * tbp_lv_area_perim_ratio

]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
special_cols = ['count_per_patient']
feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols

In [8]:
def read_data(path):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            lesion_size_ratio = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
            contrast_shape = pl.col('tbp_lv_deltaLBnorm') * pl.col('tbp_lv_area_perim_ratio'),
    
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        #.set_index(id_col)
    )

In [9]:
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [10]:
df_train = read_data(train_path)
df_test = read_data(test_path)
df_train, df_test = preprocess(df_train, df_test)

In [22]:
# Separate features (X) and target (y)
X = df_train.drop(columns=['target', 'patient_id'])
y = df_train['target']

# Features for the test set (exclude 'target' since it's not there)
X_test = df_test 

In [23]:
X_test = X_test.drop(columns=['isic_id'])

In [24]:
labels = y
labels_df = pd.DataFrame({'isic_id': X['isic_id'], 'label': labels})

In [25]:
X = X.drop(columns=['isic_id'])

In [18]:
# Create separate datasets for training and validation
train_image_path = "/kaggle/input/isic-2024-challenge/train-image.hdf5"

In [19]:
#transform images
# Train Transform can be tailored according to the needs. Or it can None
train_transform = A.Compose([
    A.Resize(height=224, width=224), #resize 
    A.OneOf([A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15),
             A.RandomBrightnessContrast() 
             ], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
    
])

In [50]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401059 entries, 0 to 401058
Columns: 221 entries, age_approx to onehot_46
dtypes: category(53), float64(154), int64(1), object(12), uint32(1)
memory usage: 532.8+ MB


In [61]:
X['age_approx'].dtype

dtype('float64')

In [55]:
# Create category_mapping
category_mapping = {}
for col in X.columns:
    if X[col].dtype == 'object':  # Categorical column
        unique_values = X[col].dropna().unique()  
        category_mapping[col] = {i: val for i, val in enumerate(unique_values)}
    if X[col].dtype == 'category':  # Categorical column
        unique_values = X[col].dropna().unique()  
        category_mapping[col] = {i: val for i, val in enumerate(unique_values)}

In [81]:
category_mapping

{'sex': {0: 'male', 1: 'female', 2: ''},
 'anatom_site_general': {0: 'lower extremity',
  1: 'head/neck',
  2: 'posterior torso',
  3: 'anterior torso',
  4: 'upper extremity',
  5: ''},
 'image_type': {0: 'TBP tile: close-up'},
 'tbp_tile_type': {0: '3D: white', 1: '3D: XP'},
 'tbp_lv_location': {0: 'Right Leg - Upper',
  1: 'Head & Neck',
  2: 'Torso Back Top Third',
  3: 'Torso Front Top Half',
  4: 'Right Arm - Upper',
  5: 'Left Leg - Upper',
  6: 'Torso Front Bottom Half',
  7: 'Left Arm - Upper',
  8: 'Right Leg',
  9: 'Torso Back Middle Third',
  10: 'Right Arm - Lower',
  11: 'Right Leg - Lower',
  12: 'Left Leg - Lower',
  13: 'Left Arm - Lower',
  14: 'Unknown',
  15: 'Left Leg',
  16: 'Torso Back Bottom Third',
  17: 'Left Arm',
  18: 'Right Arm',
  19: 'Torso Front',
  20: 'Torso Back'},
 'tbp_lv_location_simple': {0: 'Right Leg',
  1: 'Head & Neck',
  2: 'Torso Back',
  3: 'Torso Front',
  4: 'Right Arm',
  5: 'Left Leg',
  6: 'Left Arm',
  7: 'Unknown'},
 'attribution': 

In [79]:
# Function to convert a row of imputed metadata into text
def metadata_to_text(row, column_names, category_mapping=None):  # Pass column_names
    text = ""
    for col in column_names:  # Iterate over columns in X_imputed
        if X[col].dtype in ['category', 'object'] and category_mapping:  # Categorical feature
            value = row[col]
            if not pd.isna(value):
                text += f"{col}: {category_mapping[col][value]} "
            
        if X[col].dtype in ['float64', 'int64', 'uint32'] and category_mapping:
            value = row[col]
            if not pd.isna(value):
                text += f"{col}: {value:.3f} "  # Format numeric values to 2 decimal places
    return text.strip()  # Remove trailing space

In [68]:
X['age_approx'].dtype

dtype('float64')

In [80]:
# Apply the conversion to create text representations
X_text = X.apply(lambda row: metadata_to_text(row, X.columns, category_mapping), axis=1)
X_test_text = X_test.apply(lambda row: metadata_to_text(row, X_test.columns, category_mapping), axis=1)  

KeyError: 'male'

In [ ]:
# Tokenize text
train_encodings = tokenizer(X_text.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test_text.tolist(), truncation=True, padding=True)

In [ ]:
class ISICDataset(Dataset):
    def __init__(self, image_path, encodings, labels_df, transform=None):
        self.image_path = image_path
        self.encodings = encodings 
        self.transform = transform
        self.labels_df = labels_df

        with h5py.File(image_path, 'r') as f:
            all_isic_ids = list(f.keys())  # Get all IDs from the HDF5 file

            # Filter isic_ids to match the ones in your metadata
            if self.labels_df is None:
                self.isic_ids = [isic_id for isic_id in all_isic_ids] 
            else:
                self.isic_ids = [isic_id for isic_id in all_isic_ids if isic_id in undersampled_isic_ids]  
           

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        isic_id = self.isic_ids[idx]
        #print(f'Isic_ID {isic_id}')

        # Load Image
        with h5py.File(self.image_path, 'r') as f:
            image_bytes = f[isic_id][()]
        img = Image.open(io.BytesIO(image_bytes))

        # Apply Transformations
        if self.transform:
            transformed = self.transform(image=np.array(img))
            image = transformed['image']

        # Get Text Embeddings and convert to tensors
        input_ids = torch.tensor(self.encodings['input_ids'][idx]) 
        #print(f'Input Ids {input_ids}')
        attention_mask = torch.tensor(self.encodings['attention_mask'][idx])

        # Get Label and convert to tensor
        if self.labels_df is None:
            label = torch.tensor(-1)  # Or any placeholder value you prefer
        else:
            label = torch.tensor(self.labels_df[self.labels_df['isic_id'] == isic_id]['label'].values[0])

        if self.labels_df is None:
            return image, input_ids, attention_mask
        else:
            return image, input_ids, attention_mask, label 

# Create the Dataset (pass in your train_encodings)
dataset = ISICDataset(train_image_path, train_encodings, labels_df, transform=train_transform)

train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 4, pin_memory = True) 
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers = 4, pin_memory =True)

In [ ]:
print(len(dataset))  # Length of your ISICDataset
print(len(train_encodings['input_ids'])) 

In [ ]:
class MultimodalClassifier(torch.nn.Module):
    def __init__(self, num_classes=2, bert_model_path='/kaggle/input/bert-base',
                 resnet_model_path='/kaggle/input/resnet/Resnet18_Pretrained.pth', freeze_bert=False):
        super(MultimodalClassifier, self).__init__()
        # BERT for text encoding
        self.bert = BertModel.from_pretrained(bert_model_path)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        self.bert_dropout = torch.nn.Dropout(0.1)

        # Image encoder
        self.image_encoder = torch.load(resnet_model_path)
        self.image_dropout = torch.nn.Dropout(0.1)

        # Fusion layer - concatenate BERT and image embeddings
        hidden_size_bert = self.bert.config.hidden_size
        hidden_size_image = self.image_encoder.fc.in_features 

        # Calculate the combined size WITHOUT metadata
        self.combined_features_size = hidden_size_bert + 1000 #768 (Bert) + 1000 (image)
        #print(f"Combined features size: {self.combined_features_size}")  # Add this for debugging

        self.fc = torch.nn.Linear(self.combined_features_size, num_classes)

    def forward(self, input_ids, attention_mask, image_input):
        # Text encoding 
        text_input = {'input_ids': input_ids, 'attention_mask': attention_mask}
        bert_output = self.bert(**text_input)
        bert_pooled_output = bert_output.pooler_output
        #print(bert_pooled_output.shape)  # Print the shape to verify
        text_features = self.bert_dropout(bert_pooled_output)

        # Image encoding 
        image_features = self.image_encoder(image_input)
        image_features = self.image_dropout(image_features)
        #print(f"Image features shape: {image_features.shape}")  # Print shape

        

        # Combine features (concatenate only text and image)
        combined_features = torch.cat((text_features, image_features), dim=1)
        
        # Classifier
        logits = self.fc(combined_features)
        return logits


In [ ]:
# Create your MultimodalClassifier model
model = MultimodalClassifier(num_classes=1, bert_model_path='/kaggle/input/bert-base',
                             resnet_model_path='/kaggle/input/resnet/Resnet18_Pretrained.pth',
                             freeze_bert=False).to(device)

In [ ]:
# Wrap your model with DataParallel
model = DataParallel(model, device_ids=[0, 1])

In [ ]:

def score(solution: pd.DataFrame, submission: pd.DataFrame, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:torch.Size([32, 768])
taframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    

    # check submission is numeric
    if not pd.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    return(partial_auc)

In [ ]:
def p_auc(y_true, y_pred, min_tpr = 0.80):
    df = pd.DataFrame({'labels': y_true, 'predictions': y_pred})
    p_auc_score = score(df[['labels']], df[['predictions']])
    return p_auc_score

In [ ]:
# Loss function and optimizer (you might want to adjust the learning rate)
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([20]).to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate if needed
scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=0)
patience = 30

In [ ]:
num_epochs = 100

In [ ]:
torch.cuda.empty_cache() # Empty the CUDA cache

In [ ]:
#Training Loop
epochs_without_improvement = 0
# Lists to store metrics and best model/predictions
train_losses = []
train_aucs = []
val_losses = []
val_aucs = []
val_paucs = []
best_val_pauc = -1  # Initialize with a low value
best_model_state = None
best_val_preds = []  # Store predictions from the best model

for epoch in range(num_epochs):
    model.train()
    train_epoch_loss = 0
    train_preds = []
    train_true = []

    for images, input_ids, attention_mask, labels in train_dataloader:
        images, input_ids, attention_mask, labels = images.to(device), input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask, images)  # Pass BERT embeddings and images directly

        loss = criterion(logits.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        train_epoch_loss += loss.item()
        train_preds.extend(logits.sigmoid().detach().cpu().numpy().flatten())
        train_true.extend(labels.cpu().numpy())

    train_loss = train_epoch_loss / len(train_dataloader)
    train_auc = roc_auc_score(train_true, train_preds)
    train_losses.append(train_loss)
    train_aucs.append(train_auc)



    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_epoch_loss = 0
    val_preds = []
    val_true = []

    with torch.no_grad():  # No gradients needed for validation
        for images, input_ids, attention_mask, labels in val_loader:
            images, input_ids, attention_mask, labels = images.to(device), input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask, images)
            loss = criterion(outputs.squeeze(), labels.float())
            val_epoch_loss += loss.item()
            val_preds.extend(outputs.sigmoid().detach().cpu().numpy().flatten())
            val_true.extend(labels.cpu().numpy())

    val_loss = val_epoch_loss / len(val_loader)
    val_auc = roc_auc_score(val_true, val_preds)
    val_pauc = p_auc(val_true, val_preds)  # Calculate pAUC using your custom function
    val_paucs.append(val_pauc)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train AUC: {train_auc:.4f}, Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}, Val pAUC: {val_pauc:.4f}")

    scheduler.step()


    # Early Stopping Logic
    if val_pauc > best_val_pauc:
        best_val_pauc = val_pauc
        best_model_state = model.state_dict()
        epochs_without_improvement = 0  # Reset the counter


    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# After all epochs:
# Load the best model
print(f"Training Complete Best Model state loaded")
model.load_state_dict(best_model_state)
#cmd line to monitor gpu performance: watch -n 0.1 /opt/rocm/bin/rocm-smi

In [ ]:
torch.save(model, 'multimodal_15AUG_Balanced.pth')

In [ ]:
#Load my pretrained multimodal model 
model = torch.load('multimodal_15AUG_Balanced.pth')

In [ ]:
# Create Test Set DataLoader
test_dataset = ISICDataset("/kaggle/input/isic-2024-challenge/test-image.hdf5", test_encodings, labels_df=None, transform=train_transform)  # Pass isic_ids
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 4, pin_memory = True)

In [ ]:
#set the model to evaluation mode
model.eval()  # Set the model to evaluation mode

In [ ]:
# Inference Loop with Progress Bar and Prediction Collection
test_preds = []  
isic_ids = test_meta['isic_id'].tolist()  # Extract all isic_ids
#print(isic_ids)

with torch.no_grad():
    for images, input_ids, attention_mask in test_loader:  # Unpack all elements from the test_loader
        images, input_ids, attention_mask = images.to(device), input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids, attention_mask, images)
        probabilities = outputs.sigmoid().cpu().numpy().flatten()
        test_preds.extend(probabilities)


# Create DataFrame and Save as CSV
results_df = pd.DataFrame({'isic_id': isic_ids, 'target': test_preds})
#results_df.to_csv("submission.csv", index=False)
results_df.to_csv(os.path.join(working_dir,"submission.csv"), index=False)